# EGS master catalogue: Flags

In [ ]:
import numpy as np
from astropy.table import Table

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [ ]:
SUFFIX = "20180501"
FIELD = "EGS"
catname = "../../dmu1/dmu1_ml_EGS/data/master_catalogue_egs_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [ ]:
u_bands = ["Megacam u",                                  "MMT u"]
g_bands = ["Megacam g", "SUPRIME g", "GPC1 g", "BASS g", "MMT g"] 
r_bands = ["Megacam r", "SUPRIME r", "GPC1 r", "BASS r"]
i_bands = ["Megacam i", "SUPRIME i", "GPC1 i",           "MMT i"]
z_bands = ["Megacam z", "SUPRIME z", "GPC1 z", "BASS z", "MMT z"]
y_bands = [             "SUPRIME y", "GPC1 y"]
J_bands = [                                             "WIRCAM J",  "WIRCS J",                "UKIDSS J", "OMEGA2000 J"]
H_bands = [                                             "WIRCAM H"]
K_bands = [                                             "WIRCAM Ks", "WIRCS K", "NEWFIRM K",              "OMEGA2000 Ks"]

all_bands = [u_bands, g_bands, r_bands, i_bands, z_bands, y_bands, J_bands, K_bands]

In [ ]:
# No aperture magnitude -> add empty columns
for mag in ["MMT u", "MMT g", "MMT i", "MMT z", "WIRCS J", "OMEGA2000 J", "OMEGA2000 Ks"]:
    basecol = mag.replace(" ", "_").lower()
    m_ap_col, merr_ap_col = "m_ap_{}".format(basecol), "merr_ap_{}".format(basecol)
    
    master_catalogue[m_ap_col] = [np.nan] * len(master_catalogue)
    master_catalogue[merr_ap_col] = [np.nan] * len(master_catalogue)

## 1. Magnitudes and magnitude erros

In [ ]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [ ]:
## dmu0: Pan-STARRS forced photometry cat 
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_{}_ap".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

### 2.d WIRCS Aperture and Total K-band magnitudes

In [ ]:
wircs_mag = 100.0
bands = ["WIRCS K"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    col_ap, col_tot = "m_ap_{}".format(basecol), "m_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_{}_ap".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[col_ap] >= wircs_mag)[0]
    mask_tot  = np.where(master_catalogue[col_tot] >= wircs_mag)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)

## 2. Outliers

In [ ]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
        ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
        flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
        
        master_catalogue = flag_outliers(master_catalogue, col1, col2,
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
                      
                      
        
        # Tot mag
        col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
        ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
        flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
                      
        master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (total)".format(band1), "{} (total)".format(band2)))            
                      

## 3. Save table

In [ ]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)